In [9]:
import pandas as pd
import numpy as np
import pickle
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.model_selection import KFold, train_test_split
import time

In [17]:
# MIMIC-III Baseline Model Training: TF-IDF + Logistic Regression with Cross Validation


# 1. Load the preprocessed data
print("Loading preprocessed MIMIC-III data...")
data_path = '../data/'
processed_data_file = os.path.join(data_path, 'mimic3_data.pkl')
data = pd.read_pickle(processed_data_file)

print(f"Loaded {len(data)} records")

# 2. Create train and test sets (80% train, 20% test)
print("Creating train and test splits...")
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
print(f"Created splits: {len(train_data)} train, {len(test_data)} test")

# 3. Process ICD codes - build a set of all unique codes
# Modified code to handle mixed data types in ICD codes
print("Processing ICD codes...")
all_codes = set()
for codes in data['ICD9_CODE']:
    # Convert any non-string codes to strings and filter out NaN values
    valid_codes = [str(code) for code in codes if pd.notna(code)]
    all_codes.update(valid_codes)

# Now all codes should be strings, so sorting will work
code_to_idx = {code: i for i, code in enumerate(sorted(all_codes))}
idx_to_code = {i: code for code, i in code_to_idx.items()}

num_codes = len(code_to_idx)
print(f"Total unique ICD-9 codes: {num_codes}")


# 4. Create features and labels
def prepare_data(dataset):
    texts = dataset['TEXT'].tolist()
    
    # Convert ICD codes to multi-hot encoded vectors
    labels = np.zeros((len(dataset), num_codes), dtype=np.int8)
    for i, codes in enumerate(dataset['ICD9_CODE']):
        for code in codes:
            if code in code_to_idx:  # Just in case there's an unknown code
                labels[i, code_to_idx[code]] = 1
    
    return texts, labels

print("Preparing data...")
train_texts, train_labels = prepare_data(train_data)
test_texts, test_labels = prepare_data(test_data)

# 5. Create TF-IDF features
print("Creating TF-IDF features...")
start_time = time.time()
tfidf = TfidfVectorizer(
    max_features=10000,  # Limit features to reduce dimensionality
    min_df=5,            # Ignore terms that appear in less than 5 documents
    max_df=0.5,          # Ignore terms that appear in more than 50% of documents
    ngram_range=(1, 2),  # Use unigrams and bigrams
    stop_words='english' # Remove English stop words
)

train_features = tfidf.fit_transform(train_texts)
test_features = tfidf.transform(test_texts)

print(f"TF-IDF features shape: {train_features.shape}")
print(f"TF-IDF processing time: {time.time() - start_time:.2f} seconds")




Loading preprocessed MIMIC-III data...
Loaded 52726 records
Creating train and test splits...
Created splits: 42180 train, 10546 test
Processing ICD codes...
Total unique ICD-9 codes: 6918
Preparing data...
Creating TF-IDF features...
TF-IDF features shape: (42180, 10000)
TF-IDF processing time: 65.11 seconds


In [ ]:
# 6. Cross-validation and model training
print("Setting up cross-validation...")
n_folds = 5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

cv_results = {
    'fold': [],
    'micro_f1': [],
    'macro_f1': [],
    'micro_precision': [],
    'macro_precision': [],
    'micro_recall': [],
    'macro_recall': []
}

def evaluate(model, features, labels):
    # Get predictions
    pred_probs = model.predict_proba(features)
    
    # Apply threshold of 0.5 for binary classification
    preds = np.zeros_like(labels)
    for i in range(num_codes):
        # Check if this classifier exists (it may not if there was only one class for this code)
        if i < len(pred_probs) and len(pred_probs[i]) > 0:
            preds[:, i] = (pred_probs[i][:, 1] >= 0.5).astype(int)
    
    # Calculate micro and macro F1 scores
    micro_f1 = f1_score(labels, preds, average='micro', zero_division=0)
    macro_f1 = f1_score(labels, preds, average='macro', zero_division=0)
    
    # Calculate micro and macro precision and recall
    micro_precision = precision_score(labels, preds, average='micro', zero_division=0)
    macro_precision = precision_score(labels, preds, average='macro', zero_division=0)
    micro_recall = recall_score(labels, preds, average='micro', zero_division=0)
    macro_recall = recall_score(labels, preds, average='macro', zero_division=0)
    
    return {
        'micro_f1': micro_f1,
        'macro_f1': macro_f1,
        'micro_precision': micro_precision,
        'macro_precision': macro_precision,
        'micro_recall': micro_recall,
        'macro_recall': macro_recall
    }

print("Performing cross-validation...")
fold = 1
best_model = None
best_micro_f1 = 0

for train_idx, val_idx in kf.split(train_features):
    print(f"\nTraining fold {fold}/{n_folds}...")
    
    # Get fold data
    X_train_fold, X_val_fold = train_features[train_idx], train_features[val_idx]
    y_train_fold, y_val_fold = train_labels[train_idx], train_labels[val_idx]
    
    # Create a list to store estimators
    estimators = []
    trained_indices = []
    
    # Train a separate model for each ICD code that has both positive and negative examples
    for i in range(num_codes):
        # Check if there are both positive and negative examples for this code
        if np.sum(y_train_fold[:, i] == 0) > 0 and np.sum(y_train_fold[:, i] == 1) > 0:
            print(f"  Training classifier for code {i+1}/{num_codes}", end='\r')
            estimator = LogisticRegression(
                C=1.0,
                solver='saga',
                penalty='l1',
                max_iter=500,
                n_jobs=2, 
                verbose=0,
                random_state=42
            )
            try:
                estimator.fit(X_train_fold, y_train_fold[:, i])
                estimators.append(estimator)
                trained_indices.append(i)
            except Exception as e:
                print(f"  Error training classifier for code {i}: {e}")
        else:
            # Skip codes without both positive and negative examples
            pass
    
    print(f"  Trained {len(estimators)}/{num_codes} classifiers")
    
    # Create a model object to hold the estimators
    model = MultiOutputClassifier(LogisticRegression())
    model.estimators_ = estimators
    model.classes_ = [np.array([0, 1]) for _ in estimators]
    model.trained_indices = trained_indices  # Store which indices were trained
    
    # Evaluate on validation set
    metrics = evaluate(model, X_val_fold, y_val_fold)
    
    print(f"Fold {fold} validation results:")
    print(f"  Micro F1: {metrics['micro_f1']:.4f}")
    print(f"  Macro F1: {metrics['macro_f1']:.4f}")
    
    # Save metrics
    cv_results['fold'].append(fold)
    for key, value in metrics.items():
        cv_results[key].append(value)
    
    # Keep track of best model
    if metrics['micro_f1'] > best_micro_f1:
        best_micro_f1 = metrics['micro_f1']
        best_model = model
        print(f"  New best model found with micro F1: {best_micro_f1:.4f}")
    
    fold += 1

Setting up cross-validation...
Performing cross-validation...

Training fold 1/5...


/opt/anaconda3/envs/ribonn/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


/opt/anaconda3/envs/ribonn/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


/opt/anaconda3/envs/ribonn/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


/opt/anaconda3/envs/ribonn/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


...

/opt/anaconda3/envs/ribonn/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


/opt/anaconda3/envs/ribonn/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


/opt/anaconda3/envs/ribonn/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


/opt/anaconda3/envs/ribonn/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


/opt/anaconda3/envs/ribonn/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:348: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
# 7. Final model training and evaluation
print("\nTraining final model on all training data...")
start_time = time.time()

# Create a list to store estimators
final_estimators = []
trained_indices = []

# Train a separate model for each ICD code that has both positive and negative examples
for i in range(num_codes):
    # Check if there are both positive and negative examples for this code
    if np.sum(train_labels[:, i] == 0) > 0 and np.sum(train_labels[:, i] == 1) > 0:
        print(f"Training classifier for code {i+1}/{num_codes}", end='\r')
        estimator = LogisticRegression(
            C=1.0,
            solver='saga',
            penalty='l1',
            max_iter=100,
            n_jobs=1,  # Set to 1 for individual estimator
            verbose=0,
            random_state=42
        )
        try:
            estimator.fit(train_features, train_labels[:, i])
            final_estimators.append(estimator)
            trained_indices.append(i)
        except Exception as e:
            print(f"Error training classifier for code {i}: {e}")
    else:
        # Skip codes without both positive and negative examples
        pass

print(f"Trained {len(final_estimators)}/{num_codes} classifiers")

# Create a model object to hold the estimators
final_model = MultiOutputClassifier(LogisticRegression())
final_model.estimators_ = final_estimators
final_model.classes_ = [np.array([0, 1]) for _ in final_estimators]
final_model.trained_indices = trained_indices  # Store which indices were trained

print(f"Final model training time: {time.time() - start_time:.2f} seconds")

In [ ]:


# Evaluate on training set
train_metrics = evaluate(final_model, train_features, train_labels)
print("\nTraining set results:")
for key, value in train_metrics.items():
    print(f"  {key}: {value:.4f}")

# Evaluate on test set
test_metrics = evaluate(final_model, test_features, test_labels)
print("\nTest set results:")
for key, value in test_metrics.items():
    print(f"  {key}: {value:.4f}")

# 8. Analyze ICD code distribution and performance
print("\nAnalyzing ICD code distribution...")

# Count occurrences of each ICD code
code_counts = np.sum(train_labels, axis=0)
# Sort ICD codes by frequency
sorted_idx = np.argsort(-code_counts)
top_codes = [idx_to_code[i] for i in sorted_idx[:20]]
top_counts = code_counts[sorted_idx[:20]]

# Plot top 20 most frequent ICD codes
plt.figure(figsize=(12, 6))
plt.bar(range(20), top_counts)
plt.xticks(range(20), top_codes, rotation=90)
plt.title('Top 20 Most Frequent ICD-9 Codes in Training Data')
plt.xlabel('ICD-9 Code')
plt.ylabel('Frequency')
plt.tight_layout()
plt.savefig(os.path.join(data_path, 'icd_distribution.png'))
plt.show()

# 9. Plot cross-validation performance
plt.figure(figsize=(10, 6))
plt.plot(cv_results['fold'], cv_results['micro_f1'], 'o-', label='Micro F1')
plt.plot(cv_results['fold'], cv_results['macro_f1'], 'o-', label='Macro F1')
plt.xlabel('Fold')
plt.ylabel('F1 Score')
plt.title('Cross-Validation Performance')
plt.legend()
plt.grid(True)
plt.savefig(os.path.join(data_path, 'cv_performance.png'))
plt.show()

# 10. Save the model and related artifacts
print("Saving model and artifacts...")
output_dir = os.path.join(data_path, 'baseline_model')
os.makedirs(output_dir, exist_ok=True)

# Save the TF-IDF vectorizer
with open(os.path.join(output_dir, 'tfidf_vectorizer.pkl'), 'wb') as f:
    pickle.dump(tfidf, f)

# Save the trained final model
with open(os.path.join(output_dir, 'lr_model.pkl'), 'wb') as f:
    pickle.dump(final_model, f)

# Save the code mappings
with open(os.path.join(output_dir, 'code_mappings.pkl'), 'wb') as f:
    pickle.dump({
        'code_to_idx': code_to_idx,
        'idx_to_code': idx_to_code
    }, f)

# Save the CV results
cv_df.to_csv(os.path.join(output_dir, 'cv_results.csv'), index=False)

# Save the evaluation metrics
metrics = {
    'train': train_metrics,
    'test': test_metrics,
    'cv': cv_df.to_dict()
}
with open(os.path.join(output_dir, 'metrics.pkl'), 'wb') as f:
    pickle.dump(metrics, f)

print("\nBaseline model training and evaluation complete!")
print(f"Model and artifacts saved to {output_dir}")

# 11. Display example predictions
print("\nExample predictions for 5 random test samples:")
import random
sample_indices = random.sample(range(len(test_texts)), 5)

# In the display example predictions section, modify the code:
for idx in sample_indices:
    # Get true labels
    true_codes = [idx_to_code[i] for i in np.where(test_labels[idx] == 1)[0]]
    
    # Get predicted probabilities for trained classifiers only
    pred_probs = np.zeros(num_codes)
    for i, estimator_idx in enumerate(final_model.trained_indices):
        pred_probs[estimator_idx] = final_model.estimators_[i].predict_proba(test_features[idx])[0][1]
    
    # Get top 5 predicted codes with highest probability
    top_pred_indices = np.argsort(-pred_probs)[:5]
    pred_codes = [(idx_to_code[i], pred_probs[i]) for i in top_pred_indices if pred_probs[i] > 0]
    
    print(f"\nSample {idx}:")
    print(f"Text snippet: {test_texts[idx][:100]}...")
    print(f"True codes ({len(true_codes)}):")
    for code in true_codes[:5]:
        print(f"  - {code}")
    if len(true_codes) > 5:
        print(f"  - ... and {len(true_codes) - 5} more")
    
    print(f"Top 5 predicted codes (with probability):")
    for code, prob in pred_codes:
        print(f"  - {code}: {prob:.4f}")
    print("-" * 50)